In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl
import numpy as np
import pandas as pd
flowdata = pd.read_excel(r'./flowdata.xlsx', sheet_name='Scenario1')
d2 = pd.read_excel(r'./flowdata.xlsx', sheet_name='Scenario2')
d3 = pd.read_excel(r'./flowdata.xlsx', sheet_name='Scenario3')
d4 = pd.read_excel(r'./flowdata.xlsx', sheet_name='Scenario5')
flowdata = flowdata.append(d2)
flowdata = flowdata.append(d3)
flowdata = flowdata.append(d4)
flowdata = flowdata[['KTB', 'KTC', 'SKB', 'SKC']] # ensure columns ordering is correct

   KTB    SKB    KTC    SKC
0    0  228.0  412.7  183.0
1    0  232.9  422.3  179.7
2    0  230.8  414.4  178.6
3    0  230.8  413.3  177.8
4    0  229.7  412.1  177.5
5    0  228.7  410.5  175.8
6    0  231.8  415.4  178.8
7    0  228.3  410.8  173.9


**Equation**: (w)(KTB) + (x)(KTC) - (y)(SKB) - (z)(SKC) = KTB + KTC - SKB - SKC
- Each set of flow values observed for KTB, KTC, SKB, SKC can be geometrically represented as a line in 4-dimensional space mapped by w, x, y, z
- By iterating through *test_values* for w, x, y, z, we find the point (w,x,y,z) that minimises the sum of the distance between (w,x,y,z) and all the lines
- Find *num_results* number of sample points that provide the *num_results* lowest sum of distances
- Each of w, x, y, z is equal to E/1+E where E is the actual flowmeter error

In [2]:
flowdata

,KTB,KTC,SKB,SKC
0,0.0,390.7,0.0,391.7
1,0.0,388.2,0.0,388.4
2,0.0,408.9,0.0,407.2
3,0.0,413.1,0.0,407.5
4,0.0,408.3,0.0,400.8
...,...,...,...,...
29,466.5,388.9,473.7,382.7
30,455.6,397.0,463.6,388.8
31,462.6,393.9,470.2,387.0
32,455.9,399.5,462.3,391.1


**Adjustable Parameters:**

In [ ]:
num_results = 10
# error upper bound, lower bound and interval (in %)
lower_bound_error = -2
upper_bound_error = 2
error_interval = 0.1

In [3]:
# initialisation
test_values = np.arange(lower_bound_error/error_interval, upper_bound_error/error_interval)/(100/error_interval)
# introduce offset to ensure that every value is unique
results_array = [[0, 0, 0, 0, 999999 - row_num] for row_num in range(num_results)]
results = pd.DataFrame(results_array, columns=['E1(%)', 'E2(%)', 'E3(%)', 'E4(%)', 'Sum of Distances'])

In [4]:
# prepare raw data
flowdata_temp = flowdata
# change the sign of SKB and SKC columns for simpler arithmetic
flowdata_temp['SKB'] *= -1
flowdata_temp['SKC'] *= -1
flowdata_array = flowdata_temp.to_numpy()

array([[   0. ,  390.7,   -0. , -391.7],
       [   0. ,  388.2,   -0. , -388.4],
       [   0. ,  408.9,   -0. , -407.2],
       [   0. ,  413.1,   -0. , -407.5],
       [   0. ,  408.3,   -0. , -400.8],
       [   0. ,  405.5,   -0. , -399.1],
       [   0. ,  394.6,   -0. , -386.4],
       [   0. ,  407. ,   -0. , -399.6],
       [   0. ,  362.8,   -0. , -355.7],
       [   0. ,  408.5,   -0. , -404.9],
       [   0. ,  407.1,   -0. , -398.6],
       [   0. ,  407.6,   -0. , -401.4],
       [   0. ,  408.5,   -0. , -402.1],
       [   0. ,  408.6,   -0. , -409.3],
       [   0. ,  412.7, -228. , -183. ],
       [   0. ,  422.3, -232.9, -179.7],
       [   0. ,  414.4, -230.8, -178.6],
       [   0. ,  413.3, -230.8, -177.8],
       [   0. ,  412.1, -229.7, -177.5],
       [   0. ,  410.5, -228.7, -175.8],
       [   0. ,  415.4, -231.8, -178.8],
       [   0. ,  410.8, -228.3, -173.9],
       [ 444.3,    0. , -245.1, -205.1],
       [ 442.8,    0. , -248.6, -202.8],
       [ 452.7, 

In [5]:
# main algorithm
for E1 in test_values:
    w = E1/(1+E1)
    for E2 in test_values:
        x = E2/(1+E2)
        for E3 in test_values:
            y = E3/(1+E3)
            for E4 in test_values:
                z = E4/(1+E4)
                sum_of_dist = 0
                for params in flowdata_array:
                    m = sum(params) # mismatch
                    dist = abs(np.dot(params, [w,x,y,z]) - m)/np.linalg.norm(params)
                    sum_of_dist += dist
                if sum_of_dist < results['Sum of Distances'].max():
                    row_to_replace = results['Sum of Distances'].idxmax()
                    results.loc[row_to_replace] = [100*E1,100*E2,100*E3,100*E4,sum_of_dist]

In [6]:
results.sort_values(by='Sum of Distances')
results

,E1 (%),E2(%),E3(%),E4(%),Sum of Distances
0,-0.6,0.9,0.7,-1.0,2.389269
1,-0.2,0.8,0.9,-1.0,2.390446
2,-0.7,0.9,0.6,-1.0,2.390207
3,-0.2,0.9,0.9,-1.0,2.390433
4,-0.5,0.9,0.7,-1.0,2.389693
5,-0.4,0.9,0.9,-1.0,2.387399
6,-0.5,0.9,0.8,-1.0,2.388333
7,-0.4,0.9,0.8,-1.0,2.388772
8,-0.3,0.9,0.9,-1.0,2.387853
9,-0.3,0.8,0.9,-1.0,2.389619
